# Pruebas básicas
Una vez vistas las clases por las que está formada gymnasium y de que trata, se van a realizar varias pruebas con un entorno, en concreto CartPole para ver el funcionamiento general de la librería y sus posibilidades. <br>

El entorno *CartPole*, cumple las siguientes características:
- **Espacio de acciones** 
    0. para mover el carro a la izquierda
    1. para mover el carro a la derecha
- **Espacio de observaciones**:
    0. Posición del carro
    1. Velocidad del carro
    2. Ángulo del palo
    3. Velocidad angular del palo
    
La misión del entorno, es mantener el palo encima del carro el máximo tiempo posible.

En esta primera prueba, las acciones que se van a tomar van a ser random, es decir, no se va a seguir ninguna política ni ningún tipo de entrenamiento. Este ejemplo, es únicamente para ver como sería una salida típica.

In [13]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() #primer reset necesario

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): #tiempo que está en ejecución el entorno
        action = env.action_space.sample() #accion random
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated: #finalizada la ejecución del anterior
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()

Episodio terminado después de 23 timesteps
Episodio terminado después de 33 timesteps
Episodio terminado después de 19 timesteps
Episodio terminado después de 29 timesteps
Episodio terminado después de 40 timesteps
Episodio terminado después de 9 timesteps
Episodio terminado después de 24 timesteps
Episodio terminado después de 38 timesteps
Episodio terminado después de 19 timesteps
Episodio terminado después de 43 timesteps
Episodio terminado después de 19 timesteps
Episodio terminado después de 22 timesteps
Episodio terminado después de 15 timesteps
Episodio terminado después de 22 timesteps
Episodio terminado después de 14 timesteps
Episodio terminado después de 21 timesteps
Episodio terminado después de 17 timesteps
Episodio terminado después de 30 timesteps
Episodio terminado después de 20 timesteps
Episodio terminado después de 10 timesteps
Tiempo medio: 23.35


Una posible política para este problema sería que el palo intentase estar centrado, ya que la meta es mantenerse el máximo tiempo posible. Para ello, si el palo es mayor que 0, se estará cayendo a la derecha por lo que habrá que mover el carro a la derecha también. Un ejemplo de implementación sería:

In [14]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() 

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): 
        if observation[2] > 0.0:
            observation, reward, terminated, truncated, info = env.step(1)
        else:
            observation, reward, terminated, truncated, info = env.step(0)

        if terminated or truncated: 
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()





Episodio terminado después de 53 timesteps
Episodio terminado después de 57 timesteps
Episodio terminado después de 39 timesteps
Episodio terminado después de 52 timesteps
Episodio terminado después de 34 timesteps
Episodio terminado después de 36 timesteps
Episodio terminado después de 36 timesteps
Episodio terminado después de 36 timesteps
Episodio terminado después de 37 timesteps
Episodio terminado después de 36 timesteps
Episodio terminado después de 35 timesteps
Episodio terminado después de 52 timesteps
Episodio terminado después de 47 timesteps
Episodio terminado después de 25 timesteps
Episodio terminado después de 51 timesteps
Episodio terminado después de 47 timesteps
Episodio terminado después de 35 timesteps
Episodio terminado después de 66 timesteps
Episodio terminado después de 35 timesteps
Episodio terminado después de 26 timesteps
Tiempo medio: 41.75


Después de esta ejecución, se puede comprobar que se duplica el tiempo medio que aguanta el palo, por lo que la política escogida sería correcta. <br> <br>

Otra forma de posible, sería que además del ángulo, también la velocidad angular (*observation[3]*) sea positiva, ya que en casos en los que este prácticamente equilibrado, se haría un movimiento que podría ser innecesario. Un ejemplo de esta política sería:

In [44]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset() 

total = 0

for episode in range(20):
    observation, info = env.reset()
    for t in range(1000): 
        if observation[2] > 0.0 and observation[3] > 0.0:
            observation, reward, terminated, truncated, info = env.step(1)
        else:
            observation, reward, terminated, truncated, info = env.step(0)

        if terminated or truncated: 
            observation, info = env.reset()
            total += (t+1)
            print("Episodio terminado después de {} timesteps".format((t+1)))
            break
            
print("Tiempo medio: {}".format(total/20))
env.close()


/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episodio terminado después de 109 timesteps
Episodio terminado después de 138 timesteps
Episodio terminado después de 223 timesteps
Episodio terminado después de 151 timesteps
Episodio terminado después de 182 timesteps
Episodio terminado después de 109 timesteps
Episodio terminado después de 159 timesteps
Episodio terminado después de 195 timesteps
Episodio terminado después de 129 timesteps
Episodio terminado después de 114 timesteps
Episodio terminado después de 190 timesteps
Episodio terminado después de 212 timesteps
Episodio terminado después de 122 timesteps
Episodio terminado después de 142 timesteps
Episodio terminado después de 119 timesteps
Episodio terminado después de 218 timesteps
Episodio terminado después de 177 timesteps
Episodio terminado después de 214 timesteps
Episodio terminado después de 166 timesteps
Episodio terminado después de 240 timesteps
Tiempo medio: 165.45


Para aplicar Q-learning a este problema continuo, sería neceario discretizarlo para poder formar la tabla. Al ser un proceso que no tiene que ver con el uso y funcionamiento de la librería, se va a aplicar este algoritmo con un problema con espacio de observaciones discreto (*Frozen Lake*), para demostrar un caso de uso de Aprendizaje Reforzado combinado con la librería.<br> <br>

Las características de este entorno son:
- **Espacio de acciones** 
    0. Moverse a la izquierda
    1. Moverse a la derecha
    2. Moverse hacia arriba
    3. Moverse hacia abajo
- **Espacio de observaciones**:
    0. Valor representado por: $fila\_actual*nfilas + columna\_actual$ (empezando ambas desde 0). Por ejemplo, la posición 4x4 sería: $3*4 + 3 = 15$

In [46]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
plt.rcParams.update({'font.size': 17})
environment = gym.make("FrozenLake-v1", is_slippery=False)
# We re-initialize the Q-table
qtable = np.zeros((environment.observation_space.n, environment.action_space.n))

# Hyperparameters
episodes = 1000        # Total number of episodes
alpha = 0.5            # Learning rate
gamma = 0.9            # Discount factor

# List of outcomes to plot
outcomes = []

print('Q-table before training:')
print(qtable)

# Training
for _ in range(episodes):
    state = environment.reset()
    done = False

    # By default, we consider our outcome to be a failure
    outcomes.append("Failure")

    # Until the agent gets stuck in a hole or reaches the goal, keep training it
    while not done:
        # Choose the action with the highest value in the current state
        if np.max(qtable[state]) > 0:
            action = np.argmax(qtable[state])

        # If there's no best action (only zeros), take a random one
        else:
            action = environment.action_space.sample()
             
        # Implement this action and move the agent in the desired direction
        new_state, reward, done, info = environment.step(action)

        # Update Q(s,a)
        qtable[state, action] = qtable[state, action] + \
                                alpha * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])
        
        # Update our current state
        state = new_state

        # If we have a reward, it means that our outcome is a success
        if reward:
            outcomes[-1] = "Success"

print()
print('===========================================')
print('Q-table after training:')
print(qtable)

# Plot outcomes
plt.figure(figsize=(12, 5))
plt.xlabel("Run number")
plt.ylabel("Outcome")
ax = plt.gca()
ax.set_facecolor('#efeeea')
plt.bar(range(len(outcomes)), outcomes, color="#0A047A", width=1.0)
plt.show()

Q-table before training:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices